In [1]:
import xlrd
import math as m

#Oggetto nodo con attributi utili per lo svolgimento dell'algoritmo
class nodo():
	def __init__(self,ide,nome,lat,lon,stato,identificativo,defi,pot,coll,adia): 
		self.ide=ide
		self.lat=lat
		self.lon=lon
		self.nome=nome
		self.stato=stato
		self.identificativo=identificativo
		self.defi=defi
		self.pot=pot
		self.coll=coll
		self.adia=adia

#Funzione che calcola la distanza tra due punti su un piano sferico fornendole le coordinate di ciascun punto
def dist2punti(latA,lonA,latB,lonB):
	Rt=6378.388
	lat_A = (m.pi*latA)/180.
	lat_B = (m.pi*latB)/180.
	lon_A = (m.pi*lonA)/180.
	lon_B = (m.pi*lonB)/180.
	dl=abs(lon_A-lon_B)

	p = m.acos((m.sin(lat_B)*m.sin(lat_A))+(m.cos(lat_B)*m.cos(lat_A)*m.cos(dl)))
	if(lon_A>0 and lon_B>0): 
		if(lon_B > lon_A): 
			p=p
		else:
			p=float('inf')
	if(lon_A<0 and lon_B<0): 
		if(lon_B > lon_A): 
			p=p
		else:
			p=float('inf')
	if(lon_A>0 and lon_B<0):
		if((lon_A+abs(lon_B))<m.pi): 
			p=float('inf')
		else:
			p=p
	if(lon_A<0 and lon_B>0):
		if((lon_B+abs(lon_A))<m.pi): 
			p=p
		else:
			p=float('inf')
	return(p*Rt) 

#Lettura database
dati=xlrd.open_workbook('City2.xlsx')
foglio=dati.sheet_by_name('Foglio1')
nR=foglio.nrows


#Imposto la città iniziale
citta_inizio="London"
inizio="1826645935"


#Creo una serie di liste che conterranno i dati utili per losvolgimento del progetto
nome=[]
lat=[]
log=[]
stato=[]
stato2=[]
pop=[]
iden=[]
pesi=[]
nodi=[]

#Riempio le liste con i dati contenuti nel database
for i in range(1,nR):
	nome.append(foglio.cell(i,0).value)
	lat.append(foglio.cell(i,2).value)
	log.append(foglio.cell(i,3).value)
	stato.append(foglio.cell(i,6).value)
	stato2.append(foglio.cell(i,7).value)
	pop.append(foglio.cell(i,9).value)
	iden.append(str(int(foglio.cell(i,10).value)))

print("Lettura dati da file terminata\n")

#Controllo che non ci siano città per le quali non sia stata definita la numerosità della popolazione, altrimenti metto a zero
for i in range(0,nR-1):
	if(pop[i]==''):
		pop[i]=0

print("Controllo popolazioni terminato\n")

#Elimino tutti i record con valori errati (non ci possono essere due città con stesse coordinate)
elim=[]
print("Città con errori\n")
for i in range(0,nR-1):
	for j in range(0,nR-1):
		if(lat[j]==lat[i] and log[j]==log[i] and iden[j]!=iden[i]):
			print(nome[j],iden[j])
			elim.append(j)
print("\n")
            
for j in elim:
	print("rimosso per coordinate coincidenti: ",iden[j])    
	nome.pop(j)
	lat.pop(j)
	log.pop(j)
	stato.pop(j)
	stato2.pop(j)
	pop.pop(j)
	iden.pop(j)
print("\n")
#Aggiorno la numerosità dopo aver pulito i dati
nR=len(nome)

#Imposto la cittàfinale come copia di quella iniziale
citta_fine=citta_inizio+"_fine"
fine="1111111111"

for i in range(0,nR):
	if(iden[i]==inizio):
		nome.append(citta_fine)
		lat.append(lat[i])
		log.append(log[i]-0.1)
		stato.append(stato[i])
		stato2.append(stato2[i])
		pop.append(pop[i])
		iden.append(fine)

#Aggiorno la numerosità con l'aggiunta di quella finale
print("Numerosità città\n")
print(len(nome))
print("\n")
nN=len(nome)

#ciclo per calcolare le distanze e quindi i pesi di ogni città (con il calcolo di questi dati l'oggetto nodo è completo)
print("Assegnazione pesi in corso...\n")
for i in range(0,nN):
	vicini=[]
	for j in range(0,nN):
		if(iden[i]!=iden[j]):
			d=dist2punti(lat[i],log[i],lat[j],log[j])
			if(len(vicini)==0):
				vicini.append([iden[j],d,pop[j],stato[i]+"-"+stato2[i],stato[j]+"-"+stato2[j]])
			else:	
				for k in range(0,len(vicini)):
					if(d<vicini[k][1]):
						vicini.insert(k,[iden[j],d,pop[j],stato[i]+"-"+stato2[i],stato[j]+"-"+stato2[j]])
						if(len(vicini)>3):					
							vicini.pop(3)
						break
					if(len(vicini)<3):
						vicini.append([iden[j],d,pop[j],stato[i]+"-"+stato2[i],stato[j]+"-"+stato2[j]])

	diz={}
	for l in range(0,len(vicini)):
		pes=2**(l+1)
		if(vicini[l][2]>200000):
			pes=pes+2
		if(vicini[l][3] != vicini[l][4]):
			pes=pes+2
		diz[vicini[l][0]]=pes 

	print("Ancora "+str(nN-i)+" citta' rimanenti" )

	pesi.append(diz)

	nodi.append(nodo(i,nome[i],lat[i],log[i],stato[i]+"-"+stato2[i],iden[i],True,float('inf'),"",pesi[i])) 
print("\n")

#Algoritmo per il calcolo del percorso minimo in un grafo
print("Calcolo del grafo utilizzando l'algoritmo Dijkstra\n")
print("Città di partenza: " + citta_inizio)
print("\n")
print("Elenco città controllate\n")

#Imposto il primo nodo del grafo
for x in nodi:
	if(x.identificativo == inizio):
		x.defi=False
		x.pot=0
		x.coll=inizio
		i=x.ide

#Partendo dal primo controllo tutti i nodi plausibili per far parte delpercorso minimo
while True:
	for x in nodi[i].adia.keys():
		for y in nodi:
			if (y.identificativo == x and y.defi ):
				somma=nodi[i].pot + nodi[i].adia[x]
				if(y.pot>somma):
					y.pot=somma
					y.coll=nodi[i].identificativo
	m=float('inf')
	for z in nodi:
		if(z.defi and z.pot<m):
			m=z.pot
			n=z.ide
	nodi[n].defi=False
	i=n
	print(nodi[i].nome)
	if(nodi[i].identificativo == fine ):
		lung=nodi[i].pot
		break

#Due liste: una per il percorsocon gli id e l'altra con i nomi delle città        
percorso=[]
percorso_nomi=[]


#Trovoil percorso minimo a ritroso
s=fine
while True:
	percorso.append(s)
	for x in nodi:
		if(x.identificativo == s):
			s=x.coll
	if(s==inizio):
		percorso.append(s)
		break

#Converto gli id in nomi per l'output
for x in percorso:
	for y in nodi:
		if(y.identificativo==x):
			percorso_nomi.append(y.nome+" ("+y.stato+")")

#Utilizzo una stringa per contenere le tappe del percorso inserite dalla partenza all'arrivo            
strada=""
for i in range(0,len(percorso_nomi)-1):
	strada=strada+percorso_nomi[len(percorso_nomi)-1-i]+" - "
strada=strada+" "+percorso_nomi[0]

#Stampo il risultato con tutte le tappe del percorso minimo, i giorni e le ore impiegate per completarlo
print("-------------------Risultato----------------------------")
print("Percorso terminato in "+str(lung)+" ore o "+str(float(lung)/24.)+" giorni. (tempo minimo)" )
print("Tragitto: " + strada)
print("--------------------------------------------------------")

Lettura dati da file terminata

Controllo popolazioni terminato

Città con errori

San Pedro Garza García 1484228590
Chicoloapan 1484980825
Kallithéa 1300675902
Chimalhuacán 1484504936
Monterrey 1484559591
Piraeus 1300458367


rimosso per coordinate coincidenti:  1484228590
rimosso per coordinate coincidenti:  1484980825
rimosso per coordinate coincidenti:  1418863956
rimosso per coordinate coincidenti:  1484504936
rimosso per coordinate coincidenti:  1484559591
rimosso per coordinate coincidenti:  1076190259


Numerosità città

4995


Assegnazione pesi in corso...

Ancora 4995 citta' rimanenti
Ancora 4994 citta' rimanenti
Ancora 4993 citta' rimanenti
Ancora 4992 citta' rimanenti
Ancora 4991 citta' rimanenti
Ancora 4990 citta' rimanenti
Ancora 4989 citta' rimanenti
Ancora 4988 citta' rimanenti
Ancora 4987 citta' rimanenti
Ancora 4986 citta' rimanenti
Ancora 4985 citta' rimanenti
Ancora 4984 citta' rimanenti
Ancora 4983 citta' rimanenti
Ancora 4982 citta' rimanenti
Ancora 4981 citta' ri

Ancora 4722 citta' rimanenti
Ancora 4721 citta' rimanenti
Ancora 4720 citta' rimanenti
Ancora 4719 citta' rimanenti
Ancora 4718 citta' rimanenti
Ancora 4717 citta' rimanenti
Ancora 4716 citta' rimanenti
Ancora 4715 citta' rimanenti
Ancora 4714 citta' rimanenti
Ancora 4713 citta' rimanenti
Ancora 4712 citta' rimanenti
Ancora 4711 citta' rimanenti
Ancora 4710 citta' rimanenti
Ancora 4709 citta' rimanenti
Ancora 4708 citta' rimanenti
Ancora 4707 citta' rimanenti
Ancora 4706 citta' rimanenti
Ancora 4705 citta' rimanenti
Ancora 4704 citta' rimanenti
Ancora 4703 citta' rimanenti
Ancora 4702 citta' rimanenti
Ancora 4701 citta' rimanenti
Ancora 4700 citta' rimanenti
Ancora 4699 citta' rimanenti
Ancora 4698 citta' rimanenti
Ancora 4697 citta' rimanenti
Ancora 4696 citta' rimanenti
Ancora 4695 citta' rimanenti
Ancora 4694 citta' rimanenti
Ancora 4693 citta' rimanenti
Ancora 4692 citta' rimanenti
Ancora 4691 citta' rimanenti
Ancora 4690 citta' rimanenti
Ancora 4689 citta' rimanenti
Ancora 4688 ci

Ancora 4438 citta' rimanenti
Ancora 4437 citta' rimanenti
Ancora 4436 citta' rimanenti
Ancora 4435 citta' rimanenti
Ancora 4434 citta' rimanenti
Ancora 4433 citta' rimanenti
Ancora 4432 citta' rimanenti
Ancora 4431 citta' rimanenti
Ancora 4430 citta' rimanenti
Ancora 4429 citta' rimanenti
Ancora 4428 citta' rimanenti
Ancora 4427 citta' rimanenti
Ancora 4426 citta' rimanenti
Ancora 4425 citta' rimanenti
Ancora 4424 citta' rimanenti
Ancora 4423 citta' rimanenti
Ancora 4422 citta' rimanenti
Ancora 4421 citta' rimanenti
Ancora 4420 citta' rimanenti
Ancora 4419 citta' rimanenti
Ancora 4418 citta' rimanenti
Ancora 4417 citta' rimanenti
Ancora 4416 citta' rimanenti
Ancora 4415 citta' rimanenti
Ancora 4414 citta' rimanenti
Ancora 4413 citta' rimanenti
Ancora 4412 citta' rimanenti
Ancora 4411 citta' rimanenti
Ancora 4410 citta' rimanenti
Ancora 4409 citta' rimanenti
Ancora 4408 citta' rimanenti
Ancora 4407 citta' rimanenti
Ancora 4406 citta' rimanenti
Ancora 4405 citta' rimanenti
Ancora 4404 ci

Ancora 4155 citta' rimanenti
Ancora 4154 citta' rimanenti
Ancora 4153 citta' rimanenti
Ancora 4152 citta' rimanenti
Ancora 4151 citta' rimanenti
Ancora 4150 citta' rimanenti
Ancora 4149 citta' rimanenti
Ancora 4148 citta' rimanenti
Ancora 4147 citta' rimanenti
Ancora 4146 citta' rimanenti
Ancora 4145 citta' rimanenti
Ancora 4144 citta' rimanenti
Ancora 4143 citta' rimanenti
Ancora 4142 citta' rimanenti
Ancora 4141 citta' rimanenti
Ancora 4140 citta' rimanenti
Ancora 4139 citta' rimanenti
Ancora 4138 citta' rimanenti
Ancora 4137 citta' rimanenti
Ancora 4136 citta' rimanenti
Ancora 4135 citta' rimanenti
Ancora 4134 citta' rimanenti
Ancora 4133 citta' rimanenti
Ancora 4132 citta' rimanenti
Ancora 4131 citta' rimanenti
Ancora 4130 citta' rimanenti
Ancora 4129 citta' rimanenti
Ancora 4128 citta' rimanenti
Ancora 4127 citta' rimanenti
Ancora 4126 citta' rimanenti
Ancora 4125 citta' rimanenti
Ancora 4124 citta' rimanenti
Ancora 4123 citta' rimanenti
Ancora 4122 citta' rimanenti
Ancora 4121 ci

Ancora 3867 citta' rimanenti
Ancora 3866 citta' rimanenti
Ancora 3865 citta' rimanenti
Ancora 3864 citta' rimanenti
Ancora 3863 citta' rimanenti
Ancora 3862 citta' rimanenti
Ancora 3861 citta' rimanenti
Ancora 3860 citta' rimanenti
Ancora 3859 citta' rimanenti
Ancora 3858 citta' rimanenti
Ancora 3857 citta' rimanenti
Ancora 3856 citta' rimanenti
Ancora 3855 citta' rimanenti
Ancora 3854 citta' rimanenti
Ancora 3853 citta' rimanenti
Ancora 3852 citta' rimanenti
Ancora 3851 citta' rimanenti
Ancora 3850 citta' rimanenti
Ancora 3849 citta' rimanenti
Ancora 3848 citta' rimanenti
Ancora 3847 citta' rimanenti
Ancora 3846 citta' rimanenti
Ancora 3845 citta' rimanenti
Ancora 3844 citta' rimanenti
Ancora 3843 citta' rimanenti
Ancora 3842 citta' rimanenti
Ancora 3841 citta' rimanenti
Ancora 3840 citta' rimanenti
Ancora 3839 citta' rimanenti
Ancora 3838 citta' rimanenti
Ancora 3837 citta' rimanenti
Ancora 3836 citta' rimanenti
Ancora 3835 citta' rimanenti
Ancora 3834 citta' rimanenti
Ancora 3833 ci

...

Ancora 396 citta' rimanenti
Ancora 395 citta' rimanenti
Ancora 394 citta' rimanenti
Ancora 393 citta' rimanenti
Ancora 392 citta' rimanenti
Ancora 391 citta' rimanenti
Ancora 390 citta' rimanenti
Ancora 389 citta' rimanenti
Ancora 388 citta' rimanenti
Ancora 387 citta' rimanenti
Ancora 386 citta' rimanenti
Ancora 385 citta' rimanenti
Ancora 384 citta' rimanenti
Ancora 383 citta' rimanenti
Ancora 382 citta' rimanenti
Ancora 381 citta' rimanenti
Ancora 380 citta' rimanenti
Ancora 379 citta' rimanenti
Ancora 378 citta' rimanenti
Ancora 377 citta' rimanenti
Ancora 376 citta' rimanenti
Ancora 375 citta' rimanenti
Ancora 374 citta' rimanenti
Ancora 373 citta' rimanenti
Ancora 372 citta' rimanenti
Ancora 371 citta' rimanenti
Ancora 370 citta' rimanenti
Ancora 369 citta' rimanenti
Ancora 368 citta' rimanenti
Ancora 367 citta' rimanenti
Ancora 366 citta' rimanenti
Ancora 365 citta' rimanenti
Ancora 364 citta' rimanenti
Ancora 363 citta' rimanenti
Ancora 362 citta' rimanenti
Ancora 361 citta' ri

Ancora 100 citta' rimanenti
Ancora 99 citta' rimanenti
Ancora 98 citta' rimanenti
Ancora 97 citta' rimanenti
Ancora 96 citta' rimanenti
Ancora 95 citta' rimanenti
Ancora 94 citta' rimanenti
Ancora 93 citta' rimanenti
Ancora 92 citta' rimanenti
Ancora 91 citta' rimanenti
Ancora 90 citta' rimanenti
Ancora 89 citta' rimanenti
Ancora 88 citta' rimanenti
Ancora 87 citta' rimanenti
Ancora 86 citta' rimanenti
Ancora 85 citta' rimanenti
Ancora 84 citta' rimanenti
Ancora 83 citta' rimanenti
Ancora 82 citta' rimanenti
Ancora 81 citta' rimanenti
Ancora 80 citta' rimanenti
Ancora 79 citta' rimanenti
Ancora 78 citta' rimanenti
Ancora 77 citta' rimanenti
Ancora 76 citta' rimanenti
Ancora 75 citta' rimanenti
Ancora 74 citta' rimanenti
Ancora 73 citta' rimanenti
Ancora 72 citta' rimanenti
Ancora 71 citta' rimanenti
Ancora 70 citta' rimanenti
Ancora 69 citta' rimanenti
Ancora 68 citta' rimanenti
Ancora 67 citta' rimanenti
Ancora 66 citta' rimanenti
Ancora 65 citta' rimanenti
Ancora 64 citta' rimanenti


Kırıkkale
Kuytun
Khasavyurt
Irkutsk
Çorum
Orsk
Amasya
Balkanabat
Makhachkala
Shihezi
Alanya
Karaman
Ereğli
Kaspiysk
Tbilisi
Yan’an Beilu
Nukus
Kırşehir
Gonbad-e Kāvūs
Rudnyy
Derbent
Yozgat
Ulan-Ude
Tokat
Darhan
Ürümqi
Samsun
Gorgān
Niğde
Bojnūrd
Fukang
Nevşehir
Rustavi
Mersin
Sumqayıt
Aksaray
Daşoguz
Urganch
Ünye
Xırdalan
Ulaanbaatar
Turpan
Sivas
Chita
Gəncə
Sabzevār
Ordu
Shāhrūd
Altay
Kayseri
Ashgabat
Nicosia
Tarsus
Manzhouli
Talas
Qūchān
Mingəçevir
Baku
Türkmenabat
Kumul
Neyshābūr
Giresun
Adana
Yakeshi
Bukhara
Dunhuang
Kozan
Genhe
Mashhad
Trabzon
Torbat-e Ḩeydarīyeh
Kadirli
Kogon Shahri
Kāshmar
Latakia
Zalantun
Yingmen
Erzincan
Ahar
Torbat-e Jām
Osmaniye
Jiayuguan
Rize
Ţarţūs
Nehe
Kahramanmaraş
Qarshi
Bingöl
Batumi
Samandağ
Longjiang
Xibeijie
Samarkand
Ardabīl
Herāt
Mary
Shahrisabz
Zhangye
Homs
Zanjān
Hatay
Wudalianchi
Hong’an
Muş
Kutaisi
Kilis
Qiqihar
Gaziantep
Erzurum
Beian
Ḩamāh
İskenderun
Maīmanah
Jizzax
Kars
Farāh
Rasht
Jinchang
Batman
Siirt
Nizip
As Salamīyah
Daqing
Wuwei
Hailu

Nellore
Muzaffarpur
Ongole
Begusarai
Saskatoon
Kharagpur
Ellore
Glendale
Machilīpatnam
Conjeeveram
Zacatecas
El Monte
La Piedad
West Covina
Orem
Compton
Alhambra
Kirtipur
Suqian
Xinpu
Shaoguan
Huainan
Haicheng
Morelia
Laixi
Ogden
Durgāpur
Mingguang
Dengtacun
Tonalá
Kulti
Puerto Vallarta
Medinīpur
Chittoor
Saharsa
Pomona
Jāmuria
Torrance
Bhīmavaram
Tepatitlán de Morelos
Tādepallegūdem
Ciudad Lázaro Cárdenas
South Gate
Hawthorne
Chennai
Ganzhou
Hefei
Chuzhou
Anshan
Fushun
Shuyangzha
Jianguang
Zapopan
Yingcheng
Torreón
Āsansol
Provo
Pencheng
Jitpur
Guadalupe
Regina
Ontario
Pālakollu
Ciudad Hidalgo
San Francisco del Rincón
Downey
Janakpur
Bellflower
Nanchang
Jiujiang
Heyuan
Tieling
Benxi
Salt Lake City
Kathmandu
Laiyang
Saltillo
Wenchang
Luofeng
Bhāgalpur
Anaheim
Barddhamān
Diaobingshancun
Uluberiya
Vellore
Rancho Cucamonga
Grand Junction
Fullerton
Navadwīp
Billings
Zihuatanejo
Lakewood
Fuzhou
Liaoyang
León de los Aldama
Winnipeg
Aguascalientes
Chaohucun
Ruiming
Zhuangyuan
Tianchang
Kaiyua

Deltona
Oshawa
Matanzas
El Progreso
North Charleston
Hatsukaichi
Pottstown
Salisbury
Phnom Penh
Rochester
Duyun
Kissimmee
Thái Nguyên
Santa Ana
Sonsonate
Đồng Hới
Imabari
La Ceiba
Estelí
Kampong Cham
Beppu
Jacksonville
Quảng Trị
Hiroshima
Allentown
Charleston
Bắc Ninh
Palm Bay
Syracuse
Cienfuegos
Comayagua
Jinotega
Dover
Saijō
Upper Darby
Baicheng
Tongren
Port St. Lucie
Myrtle Beach
Kure
Hà Tĩnh
Binghamton
Tây Ninh
Santa Tecla
Niihama
Utica
Prey Veng
Philadelphia
Hechi
Hải Dương
Huế
Wilmington
Atlantic City
Santa Clara
Higashi-Hiroshima
Matagalpa
West Palm Beach
Comayagüela
San Salvador
Quảng Hà
Albany
Thủ Dầu Một
Sancti Spíritus
Onomichi
Zhijiang
Lilan
Lakewood
Deerfield Beach
Camden
Haiphong
Chongzuo
Tegucigalpa
Poughkeepsie
Scranton
Kōchi
Boaco
Trenton
Châu Đốc
Cao Lãnh
Lạng Sơn
Ninh Bình
Hội An
Pompano Beach
Edison
Ho Chi Minh City
Liuzhou
Laibin
Fukuyama
Usulután
Ciego de Ávila
Coral Springs
Elizabeth
Ciudad Choluteca
Granada
Woodbridge
Nanning
Biên Hòa
San José
Nassau
Springfield